## Pain Relevance Classifier

In [23]:
#Download and import relevant libraries

import nltk

#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

#Import dataframe libraries
import pandas as pd
import numpy as np
import scipy

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Import models
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE

In [24]:
def get_sampled_data_smote(X, y):
    return SMOTE(sampling_strategy='auto', random_state=42).fit_resample(X, y)

In [25]:
#Store patient data in file
file = 'patient_data1.xlsx'

#Store patient data in Pandas dataframe
data = pd.read_excel(file)
data.columns = ['Pain_Relevance', 'Pain_Change', 'Raw_Text'] #Rename columns
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 3 columns):
Pain_Relevance    424 non-null object
Pain_Change       287 non-null object
Raw_Text          400 non-null object
dtypes: object(3)
memory usage: 10.1+ KB


,Pain_Relevance,Pain_Change,Raw_Text
0,yes,increase,Patient. Arrived to VDH for pain management an...
1,yes,not sure,Patient will continue on pain control regimen ...
2,yes,not sure,Patient will continue to take pain medications...
3,no,NaN,Patient received from CHC via wheelchair on ro...
4,yes,increase,Patient remains in 8-10/10 pain. PCA initiated.


In [26]:
#Display number of values for pain relevance and pain irrelevance
print("Number of pain relevant data points: ", (data['Pain_Relevance'] == 'yes').sum())
print("Number of pain irrelevant data points: ", (data['Pain_Relevance'] == 'no').sum())
print(data['Raw_Text'][0])

Number of pain relevant data points:  391
Number of pain irrelevant data points:  33
Patient. Arrived to VDH for pain management and IVF while awaiting admission. Pain was 7/10 on arrival in chest bilaterally as documented. Patient. Somewhat anxious as portrayed by irregular breathing pattern with talking however able to tolerate eating. All VSS including pulse ox at 100% on room air. Patient. Given oral valium and IV pain meds. Patient. Appears more calm after valium administration. Will transfer to 5100 via wheelchair with mother at side. ACRN


In [27]:
#Change newline character(s) in Pain Change column to "not sure"
data.loc[data['Pain_Change'] == '\n', 'Pain_Change'] = 'not sure'

#Change null Pain Change values where Pain Relevance is true to "not sure"
data.loc[(data['Pain_Relevance'] == 'yes') & (data['Pain_Change'] == np.NaN), 'Pain_Change'] = 'not sure'

#Drop rows with null Pain Relevance or null Raw Text fields
data = data.dropna(subset=['Pain_Relevance', 'Raw_Text'])
data.info()
data.head()
print("Pain relevant data points with no null values: ", (data['Pain_Relevance'] == 'yes').sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 423
Data columns (total 3 columns):
Pain_Relevance    400 non-null object
Pain_Change       280 non-null object
Raw_Text          400 non-null object
dtypes: object(3)
memory usage: 12.5+ KB
Pain relevant data points with no null values:  382


In [28]:
data.head()

,Pain_Relevance,Pain_Change,Raw_Text
0,yes,increase,Patient. Arrived to VDH for pain management an...
1,yes,not sure,Patient will continue on pain control regimen ...
2,yes,not sure,Patient will continue to take pain medications...
3,no,NaN,Patient received from CHC via wheelchair on ro...
4,yes,increase,Patient remains in 8-10/10 pain. PCA initiated.


In [29]:
#Print unique values in Pain Relevance column
print(pd.unique(data['Pain_Relevance']))
print(data['Pain_Relevance'].value_counts())

['yes' 'no']
yes    382
no      18
Name: Pain_Relevance, dtype: int64


In [30]:
#Print unique values in Pain Change column
print(pd.unique(data['Pain_Change']))
print(data['Pain_Change'].value_counts())

['increase' 'not sure' nan 'decrease' 'no change']
decrease     145
increase      51
no change     51
not sure      33
Name: Pain_Change, dtype: int64


In [31]:
#Change text to lower case, remove stopwords and punctuation
data['Raw_Text'] = data['Raw_Text'].str.lower()

stop_words = set(stopwords.words("english")) 
lem = WordNetLemmatizer()
#ps = PorterStemmer()

def preprocess(row):
    text = row['Raw_Text']
    #print(text) #Test statement
    meaningful_words = [w for w in nltk.tokenize.word_tokenize(text) if not w in stop_words]  
    #print(" ".join(meaningful_words)) #Test statement
    
    proc_words = []
    for w in meaningful_words:
        proc_words.append(lem.lemmatize(w))
        #proc_words.append(ps.stem(w))
    #print(" ".join(proc_words)) #Test statement
    return " ".join(proc_words)

data['Raw_Text'] = data.apply(preprocess, axis=1)

data.head()

,Pain_Relevance,Pain_Change,Raw_Text
0,yes,increase,patient . arrived vdh pain management ivf awai...
1,yes,not sure,patient continue pain control regimen 0/10 pain .
2,yes,not sure,patient continue take pain medication without ...
3,no,NaN,patient received chc via wheelchair room air ....
4,yes,increase,patient remains 8-10/10 pain . pca initiated .


In [32]:
data['Str_Text'] = data.Raw_Text.astype(str) 
data.head()

,Pain_Relevance,Pain_Change,Raw_Text,Str_Text
0,yes,increase,patient . arrived vdh pain management ivf awai...,patient . arrived vdh pain management ivf awai...
1,yes,not sure,patient continue pain control regimen 0/10 pain .,patient continue pain control regimen 0/10 pain .
2,yes,not sure,patient continue take pain medication without ...,patient continue take pain medication without ...
3,no,NaN,patient received chc via wheelchair room air ....,patient received chc via wheelchair room air ....
4,yes,increase,patient remains 8-10/10 pain . pca initiated .,patient remains 8-10/10 pain . pca initiated .


In [33]:
#Remove punctuation and vectorize text
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv_n1 = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
cv_n2 = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,2),tokenizer = token.tokenize)
cv_n3 = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,3),tokenizer = token.tokenize)
vectorized_data_n1 = cv_n1.fit_transform(data['Str_Text'].copy())
vectorized_data_n2 = cv_n2.fit_transform(data['Str_Text'].copy())
vectorized_data_n3 = cv_n3.fit_transform(data['Str_Text'].copy())

encoder = LabelEncoder()
data['Pain_Relevance'] = encoder.fit_transform(data['Pain_Relevance'].astype(str))

y = data[['Pain_Relevance']]
y = np.ravel(y)

#Select k best features
#Select k best features
kx1 = SelectKBest(chi2, k=10)
kx2 = SelectKBest(chi2, k=10)
kx3 = SelectKBest(chi2, k=10)


x1 = kx1.fit_transform(vectorized_data_n1, y)
x2 = kx2.fit_transform(vectorized_data_n2, y)
x3 = kx3.fit_transform(vectorized_data_n3, y)

x1.shape
x2.shape
x3.shape

(400, 10)

In [34]:
#Set test ratio and get training and testing sets
test_ratio = 0.3
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y, test_size=test_ratio, shuffle=True, random_state=1)
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y, test_size=test_ratio, shuffle=True, random_state=1)
x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y, test_size=test_ratio, shuffle=True, random_state=1)

print("Number of unigram features: ", x1_train.shape)
print("Number of bigram features: ", x2_train.shape)
print("Number of trigram features: ", x3_train.shape)

Number of unigram features:  (280, 10)
Number of bigram features:  (280, 10)
Number of trigram features:  (280, 10)


In [35]:
x1_train, y1_train = get_sampled_data_smote(x1_train, y1_train)
x2_train, y2_train = get_sampled_data_smote(x2_train, y2_train)
x3_train, y3_train = get_sampled_data_smote(x3_train, y3_train)

In [36]:
#Display all features for each n-gram model
feature_names_n1 = cv_n1.get_feature_names()
bow_features_n1 = pd.DataFrame(vectorized_data_n1.toarray(), columns = feature_names_n1)
#print(bow_features_n1)

feature_names_n2 = cv_n2.get_feature_names()
bow_features_n2 = pd.DataFrame(vectorized_data_n2.toarray(), columns = feature_names_n2)
#print(bow_features_n2)

feature_names_n3 = cv_n3.get_feature_names()
bow_features_n3 = pd.DataFrame(vectorized_data_n3.toarray(), columns = feature_names_n3)
#print(bow_features_n3)

In [37]:
#Show k-best features for each n-gram model
mask = kx1.get_support(indices=True)
new_features1 = bow_features_n1.columns[mask]
print(new_features1)

mask = kx2.get_support(indices=True)
new_features2 = bow_features_n2.columns[mask]
#print(new_features2)

mask = kx3.get_support(indices=True)
new_features3 = bow_features_n3.columns[mask]
#print(new_features3)

Index(['ability', 'admission', 'discharge', 'goal', 'home', 'note', 'outcome',
       'pain', 'progressing', 'respiratory'],
      dtype='object')


In [38]:
#Train and evaluate neural network model
mlp = MLPClassifier(hidden_layer_sizes=(5,5), max_iter=10000, random_state=42)
#mlp = MLPClassifier(hidden_layer_sizes=(5,5), max_iter=10000)
mlp = mlp.fit(x1_train, y1_train)
train_pred = mlp.predict(x1_train)
test_pred = mlp.predict(x1_test)

'''
print('Neural Network Unigram Performance on Training Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_train, train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_train, train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_train, train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_train, train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_train, train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_train, train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_train, train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_train, train_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_train, train_pred))

'''


#print()

print('Neural Network Unigram Performance on Test Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, test_pred))


print()
'''
mlp = mlp.fit(x2_train, y2_train)
train_pred = mlp.predict(x2_train)
test_pred = mlp.predict(x2_test)

print('Neural Network Bigram Performance on Training Set:')
print('Parameters: ', mlp.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_train, train_pred))

print()

print('Neural Network Bigram Performance on Testing Set:')
print('Parameters: ', mlp.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_test, test_pred))

print()


mlp = mlp.fit(x3_train, y3_train)
train_pred = mlp.predict(x3_train)
test_pred = mlp.predict(x3_test)

print('Neural Network Trigram Performance on Training Set:')
print('Parameters: ', mlp.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_train, train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_train, train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_train, train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_train, train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_train, train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_train, train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_train, train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_train, train_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_train, train_pred))

print()

print('Neural Network Trigram Performance on Testing Set:')
print('Parameters: ', mlp.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_test, test_pred))
'''


Neural Network Unigram Performance on Test Set:
Macro Precision Score: 0.688
Weighted Precision Score: 0.969
Macro Recall Score: 0.956
Weighted Recall Score: 0.917
Micro F1 Score: 0.917
Macro F1 Score: 0.750
Weighted F1 Score: 0.934
Confusion Matrix: 
 [[  6   0]
 [ 10 104]]



"\nmlp = mlp.fit(x2_train, y2_train)\ntrain_pred = mlp.predict(x2_train)\ntest_pred = mlp.predict(x2_test)\n\nprint('Neural Network Bigram Performance on Training Set:')\nprint('Parameters: ', mlp.get_params)\nprint('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, train_pred))\nprint('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, train_pred, average='macro'))\nprint('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, train_pred, average='weighted'))\nprint('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, train_pred, average='macro'))\nprint('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, train_pred, average='weighted'))\nprint('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='micro'))\nprint('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='macro'))\nprint('Weighted F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='weighted'))\n#print('Mean Accuracy Score

In [39]:
#Train and evaluate neural network model
mlp = MLPClassifier(hidden_layer_sizes=(50,50), max_iter=10000, random_state=42)
#mlp = MLPClassifier(hidden_layer_sizes=(50,50), max_iter=10000)
mlp = mlp.fit(x1_train, y1_train)
train_pred = mlp.predict(x1_train)
test_pred = mlp.predict(x1_test)

'''
print('Neural Network Performance on Training Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_train, train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_train, train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_train, train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_train, train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_train, train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_train, train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_train, train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_train, train_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_train, train_pred))

'''
#print()

print('Neural Network Performance on Testing Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, test_pred))

'''
print()

mlp = mlp.fit(x2_train, y2_train)
train_pred = mlp.predict(x2_train)
test_pred = mlp.predict(x2_test)

print('Neural Network Bigram Performance on Training Set:')
print('Parameters: ', mlp.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_train, train_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_train, train_pred))

print()
'''
print('Neural Network Bigram Performance on Testing Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_test, test_pred))

print()

mlp = mlp.fit(x3_train, y3_train)
predictions = mlp.predict(x3_test)

'''
print('Neural Network Trigram Performance on Training Set:')
print('Parameters: ', mlp.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_train, train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_train, train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_train, train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_train, train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_train, train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_train, train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_train, train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_train, train_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_train, train_pred))

print()
'''
print('Neural Network Trigram Performance on Testing Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_test, test_pred))

Neural Network Performance on Testing Set:
Macro Precision Score: 0.688
Weighted Precision Score: 0.969
Macro Recall Score: 0.956
Weighted Recall Score: 0.917
Micro F1 Score: 0.917
Macro F1 Score: 0.750
Weighted F1 Score: 0.934
Confusion Matrix: 
 [[  6   0]
 [ 10 104]]
Neural Network Bigram Performance on Testing Set:
Macro Precision Score: 0.688
Weighted Precision Score: 0.969
Macro Recall Score: 0.956
Weighted Recall Score: 0.917
Micro F1 Score: 0.917
Macro F1 Score: 0.750
Weighted F1 Score: 0.934
Confusion Matrix: 
 [[  6   0]
 [ 10 104]]

Neural Network Trigram Performance on Testing Set:
Macro Precision Score: 0.688
Weighted Precision Score: 0.969
Macro Recall Score: 0.956
Weighted Recall Score: 0.917
Micro F1 Score: 0.917
Macro F1 Score: 0.750
Weighted F1 Score: 0.934
Confusion Matrix: 
 [[  6   0]
 [ 10 104]]


In [40]:
#Train a Decision Tree Classifier on the data, make predictions, and evaluate
dt = DecisionTreeClassifier(random_state=42)
#dt = DecisionTreeClassifier()
dt = dt.fit(x1_train, y1_train)
dt_train_pred = dt.predict(x1_train)
dt_test_pred = dt.predict(x1_test)

'''
print("Decision Tree Performance on Training Set:")
print("Parameters: ", dt.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_train, dt_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_train, dt_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_train, dt_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_train, dt_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_train, dt_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_train, dt_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_train, dt_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_train, dt_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_train, dt_train_pred))

print()

'''

print("Decision Tree Performance on Testing Set:")
#print("Parameters: ", dt.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, dt_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, dt_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, dt_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, dt_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, dt_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, dt_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, dt_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, dt_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, dt_test_pred))


'''
print()

dt = dt.fit(x2_train, y2_train)
dt_train_pred = dt.predict(x2_train)
dt_test_pred = dt.predict(x2_test)

print("Decision Tree Bigram Performance on Training Set:")
print("Parameters: ", dt.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, dt_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, dt_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, dt_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, dt_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, dt_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, dt_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, dt_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_train, dt_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_train, dt_train_pred))

print()
'''
print("Decision Tree Bigram Performance on Testing Set:")
#print("Parameters: ", dt.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_test, dt_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_test, dt_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, dt_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, dt_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, dt_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_test, dt_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_test, dt_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_test, dt_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_test, dt_test_pred))


Decision Tree Performance on Testing Set:
Macro Precision Score: 0.731
Weighted Precision Score: 0.973
Macro Recall Score: 0.969
Weighted Recall Score: 0.942
Micro F1 Score: 0.942
Macro F1 Score: 0.800
Weighted F1 Score: 0.951
Confusion Matrix: 
 [[  6   0]
 [  7 107]]
Decision Tree Bigram Performance on Testing Set:
Macro Precision Score: 0.731
Weighted Precision Score: 0.973
Macro Recall Score: 0.969
Weighted Recall Score: 0.942
Micro F1 Score: 0.942
Macro F1 Score: 0.800
Weighted F1 Score: 0.951
Confusion Matrix: 
 [[  6   0]
 [  7 107]]


In [41]:
#Train a Naive Bayes Classifier on the data, make predictions, and evaluate
gnb = GaussianNB()
gnb = gnb.fit(x1_train.toarray(), y1_train)
gnb_train_pred = gnb.predict(x1_train.toarray())
gnb_test_pred = gnb.predict(x1_test.toarray())

print("Naive Bayes Performance on Training Set:")
print("Parameters: ", gnb.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_train, gnb_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_train, gnb_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_train, gnb_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_train, gnb_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_train, gnb_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_train, gnb_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_train, gnb_train_pred, average='macro'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_train, gnb_train_pred))

print()

print("Naive Bayes Performance on Testing Set:")
print("Parameters: ", gnb.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, gnb_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, gnb_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, gnb_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, gnb_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, gnb_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, gnb_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, gnb_test_pred, average='macro'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, gnb_test_pred))

print()

gnb = gnb.fit(x2_train.toarray(), y2_train)
gnb_train_pred = gnb.predict(x2_train.toarray())
gnb_test_pred = gnb.predict(x2_test.toarray())

print("Naive Bayes Bigram Performance on Training Set:")
print("Parameters: ", gnb.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, gnb_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, gnb_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, gnb_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, gnb_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, gnb_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, gnb_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, gnb_train_pred, average='macro'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_train, gnb_train_pred))

print()

print("Naive Bayes Bigram Performance on Testing Set:")
print("Parameters: ", gnb.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_test, gnb_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_test, gnb_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, gnb_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, gnb_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, gnb_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_test, gnb_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_test, gnb_test_pred, average='macro'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_test, gnb_test_pred))

Naive Bayes Performance on Training Set:
Parameters:  <bound method BaseEstimator.get_params of GaussianNB(priors=None, var_smoothing=1e-09)>
Accuracy Score: 0.927
Macro Precision Score: 0.927
Weighted Precision Score: 0.927
Macro Recall Score: 0.927
Weighted Recall Score: 0.927
Micro F1 Score: 0.927
Macro F1 Score: 0.927
Confusion Matrix: 
 [[251  17]
 [ 22 246]]

Naive Bayes Performance on Testing Set:
Parameters:  <bound method BaseEstimator.get_params of GaussianNB(priors=None, var_smoothing=1e-09)>
Accuracy Score: 0.892
Macro Precision Score: 0.658
Weighted Precision Score: 0.966
Macro Recall Score: 0.943
Weighted Recall Score: 0.892
Micro F1 Score: 0.892
Macro F1 Score: 0.710
Confusion Matrix: 
 [[  6   0]
 [ 13 101]]

Naive Bayes Bigram Performance on Training Set:
Parameters:  <bound method BaseEstimator.get_params of GaussianNB(priors=None, var_smoothing=1e-09)>
Accuracy Score: 0.612
Macro Precision Score: 0.746
Weighted Precision Score: 0.746
Macro Recall Score: 0.612
Weighte

In [42]:
#Train a Logistic Regression Classifier
param_grid_lr = {
         'C': [1.0, 1e2, 1e3, 5e3, 1e4, 5e4, 1e5],
         'solver': ('newton-cg', 'lbfgs', 'sag', 'saga')
          }
lr = GridSearchCV(LogisticRegression(random_state=42, multi_class='multinomial', max_iter=40000), param_grid_lr, cv=5)
#lr = GridSearchCV(LogisticRegression(multi_class='multinomial', max_iter=40000), param_grid_lr, cv=5)
lr = lr.fit(x1_train, y1_train)
lr_train_pred = lr.predict(x1_train)
lr_test_pred = lr.predict(x1_test)

'''
print("Logistic Regression Performance on Training Set:")
print("Parameters: ", lr.best_params_)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_train, lr_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_train, lr_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_train, lr_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_train, lr_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_train, lr_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_train, lr_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_train, lr_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_train, lr_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_train, lr_train_pred))

print()

'''

print("Logistic Regression Performance on Testing Set:")
#print("Parameters: ", lr.best_params_)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, lr_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, lr_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, lr_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, lr_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, lr_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, lr_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, lr_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, lr_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, lr_test_pred))

print()

lr = lr.fit(x2_train, y2_train)
lr_train_pred = lr.predict(x2_train)
lr_test_pred = lr.predict(x2_test)

'''
print("Logistic Regression Bigram Performance on Training Set:")
print("Parameters: ", lr.best_params_)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, lr_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, lr_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, lr_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, lr_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, lr_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, lr_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, lr_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_train, lr_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_train, lr_train_pred))

print()

'''

print("Logistic Regression Bigram Performance on Testing Set:")
#print("Parameters: ", lr.best_params_)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_test, lr_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_test, lr_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, lr_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, lr_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, lr_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_test, lr_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_test, lr_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_test, lr_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_test, lr_test_pred))

print()

lr = lr.fit(x3_train, y3_train)
lr_train_pred = lr.predict(x3_train)
lr_test_pred = lr.predict(x3_test)

'''
print("Logistic Regression Trigram Performance on Training Set:")
print("Parameters: ", lr.best_params_)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_train, lr_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_train, lr_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_train, lr_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_train, lr_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_train, lr_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_train, lr_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_train, lr_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_train, lr_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_train, lr_train_pred))

print()

'''

print("Logistic Regression Trigram Performance on Testing Set:")
#print("Parameters: ", lr.best_params_)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_test, lr_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_test, lr_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_test, lr_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_test, lr_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_test, lr_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_test, lr_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_test, lr_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_test, lr_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_test, lr_test_pred))

Logistic Regression Performance on Testing Set:
Macro Precision Score: 0.625
Weighted Precision Score: 0.963
Macro Recall Score: 0.921
Weighted Recall Score: 0.850
Micro F1 Score: 0.850
Macro F1 Score: 0.657
Weighted F1 Score: 0.889
Confusion Matrix: 
 [[ 6  0]
 [18 96]]

Logistic Regression Bigram Performance on Testing Set:
Macro Precision Score: 0.625
Weighted Precision Score: 0.963
Macro Recall Score: 0.921
Weighted Recall Score: 0.850
Micro F1 Score: 0.850
Macro F1 Score: 0.657
Weighted F1 Score: 0.889
Confusion Matrix: 
 [[ 6  0]
 [18 96]]

Logistic Regression Trigram Performance on Testing Set:
Macro Precision Score: 0.625
Weighted Precision Score: 0.963
Macro Recall Score: 0.921
Weighted Recall Score: 0.850
Micro F1 Score: 0.850
Macro F1 Score: 0.657
Weighted F1 Score: 0.889
Confusion Matrix: 
 [[ 6  0]
 [18 96]]


In [43]:
#Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42)
#rf = RandomForestClassifier(n_estimators=100, max_depth=2)
rf = rf.fit(x1_train, y1_train)
rf_train_pred = rf.predict(x1_train)
rf_test_pred = rf.predict(x1_test)

'''
print("Random Forest Performance on Training Set:")
print("Paramters: ", rf.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_train, rf_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_train, rf_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_train, rf_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_train, rf_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_train, rf_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_train, rf_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_train, rf_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_train, rf_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_train, rf_train_pred))

'''

#print()

print("Random Forest Performance on Testing Set:")
#print("Paramters: ", rf.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, rf_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, rf_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, rf_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, rf_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, rf_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, rf_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, rf_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, rf_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, rf_test_pred))

print()

rf = rf.fit(x2_train, y2_train)
rf_train_pred = rf.predict(x2_train)
rf_test_pred = rf.predict(x2_test)

'''
print("Random Forest Bigram Performance on Training Set:")
print("Paramters: ", rf.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_train, rf_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_train, rf_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_train, rf_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_train, rf_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_train, rf_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_train, rf_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_train, rf_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_train, rf_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_train, rf_train_pred))

print()

'''

print("Random Forest Bigram Performance on Testing Set:")
#print("Paramters: ", rf.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y2_test, rf_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y2_test, rf_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, rf_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, rf_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, rf_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y2_test, rf_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y2_test, rf_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y2_test, rf_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y2_test, rf_test_pred))

print()

rf = rf.fit(x3_train, y3_train)
rf_train_pred = rf.predict(x3_train)
rf_test_pred = rf.predict(x3_test)

'''
print("Random Forest Trigram Performance on Training Set:")
print("Paramters: ", rf.get_params)
print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_train, rf_train_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_train, rf_train_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_train, rf_train_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_train, rf_train_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_train, rf_train_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_train, rf_train_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_train, rf_train_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_train, rf_train_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_train, rf_train_pred))

print()

'''

print("Random Forest Trigram Performance on Testing Set:")
#print("Paramters: ", rf.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y3_test, rf_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y3_test, rf_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y3_test, rf_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y3_test, rf_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y3_test, rf_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y3_test, rf_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y3_test, rf_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y3_test, rf_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y3_test, rf_test_pred))

Random Forest Performance on Testing Set:
Macro Precision Score: 0.603
Weighted Precision Score: 0.960
Macro Recall Score: 0.899
Weighted Recall Score: 0.808
Micro F1 Score: 0.808
Macro F1 Score: 0.615
Weighted F1 Score: 0.861
Confusion Matrix: 
 [[ 6  0]
 [23 91]]

Random Forest Bigram Performance on Testing Set:
Macro Precision Score: 0.594
Weighted Precision Score: 0.959
Macro Recall Score: 0.886
Weighted Recall Score: 0.783
Micro F1 Score: 0.783
Macro F1 Score: 0.594
Weighted F1 Score: 0.844
Confusion Matrix: 
 [[ 6  0]
 [26 88]]

Random Forest Trigram Performance on Testing Set:
Macro Precision Score: 0.594
Weighted Precision Score: 0.959
Macro Recall Score: 0.886
Weighted Recall Score: 0.783
Micro F1 Score: 0.783
Macro F1 Score: 0.594
Weighted F1 Score: 0.844
Confusion Matrix: 
 [[ 6  0]
 [26 88]]


# **LDA Topics as features to ML models**

In [44]:
#LDA topics as features to ML

from nltk.corpus import stopwords
import gensim
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
#import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

stop_words = stopwords.words('english')

#list of stopwords
with open('terrier-stop.txt') as f:
    stop_words = f.read().splitlines()
    
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
                  'would','really','like','great','service','came','got'])


def strip_newline(series):
    return [str(review).replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
    
def get_corpus(df):
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

rev_train = data.copy()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/amanuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
rev_train.head()

,Pain_Relevance,Pain_Change,Raw_Text,Str_Text
0,1,increase,patient . arrived vdh pain management ivf awai...,patient . arrived vdh pain management ivf awai...
1,1,not sure,patient continue pain control regimen 0/10 pain .,patient continue pain control regimen 0/10 pain .
2,1,not sure,patient continue take pain medication without ...,patient continue take pain medication without ...
3,0,NaN,patient received chc via wheelchair room air ....,patient received chc via wheelchair room air ....
4,1,increase,patient remains 8-10/10 pain . pca initiated .,patient remains 8-10/10 pain . pca initiated .


In [46]:
rev_train = rev_train.rename(columns={"Str_Text": "text"})

In [47]:
train_corpus4, train_id2word4, bigram_train4 = get_corpus(rev_train)

In [48]:
#bigram_train4[0]

In [49]:
lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus4,
                           num_topics=2,
                           id2word=train_id2word4,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)

In [50]:
lda_train4.print_topics(2,num_words=10)

[(0,
  '0.172*"medication" + 0.145*"satisfied" + 0.141*"intervention" + 0.141*"emar" + 0.073*"pca" + 0.051*"collaborate" + 0.040*"shift" + 0.034*"toradol" + 0.032*"admission" + 0.031*"time"'),
 (1,
  '0.146*"regimen" + 0.096*"satisfaction" + 0.095*"knowledge" + 0.071*"cope_supported" + 0.052*"deficit" + 0.051*"state_carry" + 0.051*"method" + 0.045*"met_continue" + 0.043*"expression_comfortable" + 0.042*"alteration_comfort"')]

In [51]:
train_vecs = []
for i in range(len(rev_train)):
    top_topics = lda_train4.get_document_topics(train_corpus4[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(2)]
    topic_vec.extend([len(rev_train.iloc[i].text)]) # length of a clinical note
    train_vecs.append(topic_vec)

In [52]:
train_vecs[1]

[0.26196265, 0.7380373, 49]

In [53]:
X = np.array(train_vecs)

In [54]:
x1_array = csr_matrix(x1, dtype=np.int8).toarray()

In [55]:
np.concatenate((x1_array[1], X[1]), axis=0)

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  2.        ,  0.        ,  0.        ,
        0.26196265,  0.73803729, 49.        ])

In [56]:
X_cumulative = list()
for i,j in zip(x1_array,X):
    X_cumulative.append(np.concatenate((i, j), axis=0))

In [57]:
x_cum_train, x_cum_test, y1_train, y1_test = train_test_split(X_cumulative, y, test_size=test_ratio, shuffle=True, random_state=1)

# Train Classifiers with concatenated LDA topics and n-grams as features

In [58]:
#Train a Logistic Regression Classifier with concatenated LDA topics and n-grams as features
param_grid_lr = {
         'C': [1.0, 1e2, 1e3, 5e3, 1e4, 5e4, 1e5],
         'solver': ('newton-cg', 'lbfgs', 'sag', 'saga')
          }
lr = GridSearchCV(LogisticRegression(random_state=42, multi_class='multinomial', max_iter=40000), param_grid_lr, cv=5)
#lr = GridSearchCV(LogisticRegression(multi_class='multinomial', max_iter=40000), param_grid_lr, cv=5)
lr = lr.fit(x_cum_train, y1_train)
lr_train_pred = lr.predict(x_cum_train)
lr_test_pred = lr.predict(x_cum_test)


print("Logistic Regression Performance on Testing Set:")
#print("Parameters: ", lr.best_params_)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, lr_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, lr_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, lr_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, lr_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, lr_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, lr_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, lr_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, lr_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, lr_test_pred))


Logistic Regression Performance on Testing Set:
Macro Precision Score: 0.475
Weighted Precision Score: 0.902
Macro Recall Score: 0.500
Weighted Recall Score: 0.950
Micro F1 Score: 0.950
Macro F1 Score: 0.487
Weighted F1 Score: 0.926
Confusion Matrix: 
 [[  0   6]
 [  0 114]]


/home/amanuel/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
#Train a Decision Tree Classifier with concatenated LDA topics and n-grams as features
dt = DecisionTreeClassifier(random_state=42)
#dt = DecisionTreeClassifier()
dt = dt.fit(x_cum_train, y1_train)
dt_train_pred = dt.predict(x_cum_train)
dt_test_pred = dt.predict(x_cum_test)


print("Decision Tree Performance on Testing Set:")
#print("Parameters: ", dt.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, dt_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, dt_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, dt_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, dt_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, dt_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, dt_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, dt_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, dt_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, dt_test_pred))

Decision Tree Performance on Testing Set:
Macro Precision Score: 0.773
Weighted Precision Score: 0.977
Macro Recall Score: 0.978
Weighted Recall Score: 0.958
Micro F1 Score: 0.958
Macro F1 Score: 0.842
Weighted F1 Score: 0.964
Confusion Matrix: 
 [[  6   0]
 [  5 109]]


In [60]:
#Train a Random Forest Classifier with concatenated LDA topics and n-grams as features

rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42)
#rf = RandomForestClassifier(n_estimators=100, max_depth=2)
rf = rf.fit(x_cum_train, y1_train)
rf_train_pred = rf.predict(x_cum_train)
rf_test_pred = rf.predict(x_cum_test)


print()

print("Random Forest Performance on Testing Set:")
#print("Paramters: ", rf.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, rf_test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, rf_test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, rf_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, rf_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, rf_test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, rf_test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, rf_test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, rf_test_pred, average='weighted'))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, rf_test_pred))



Random Forest Performance on Testing Set:
Macro Precision Score: 0.475
Weighted Precision Score: 0.902
Macro Recall Score: 0.500
Weighted Recall Score: 0.950
Micro F1 Score: 0.950
Macro F1 Score: 0.487
Weighted F1 Score: 0.926
Confusion Matrix: 
 [[  0   6]
 [  0 114]]


/home/amanuel/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
#Train an MLP Classifier with concatenated LDA topics and n-grams as features

mlp = MLPClassifier(hidden_layer_sizes=(5,5), max_iter=10000, random_state=42)
#mlp = MLPClassifier(hidden_layer_sizes=(5,5), max_iter=10000)
mlp = mlp.fit(x_cum_train, y1_train)
train_pred = mlp.predict(x_cum_train)
test_pred = mlp.predict(x_cum_test)



#print()

print('Neural Network Unigram Performance on Test Set:')
#print('Parameters: ', mlp.get_params)
#print('Accuracy Score: %.3f' % metrics.accuracy_score(y1_test, test_pred))
print('Macro Precision Score: %.3f' % metrics.precision_score(y1_test, test_pred, average='macro'))
print('Weighted Precision Score: %.3f' % metrics.precision_score(y1_test, test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y1_test, test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y1_test, test_pred, average='weighted'))
print('Micro F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='micro'))
print('Macro F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='macro'))
print('Weighted F1 Score: %.3f' % metrics.f1_score(y1_test, test_pred, average='weighted'))
#print('Mean Accuracy Score: %.3f' % mlp.score(x_test, y_test))
print('Confusion Matrix: \n', metrics.confusion_matrix(y1_test, test_pred))



Neural Network Unigram Performance on Test Set:
Macro Precision Score: 0.737
Weighted Precision Score: 0.950
Macro Recall Score: 0.737
Weighted Recall Score: 0.950
Micro F1 Score: 0.950
Macro F1 Score: 0.737
Weighted F1 Score: 0.950
Confusion Matrix: 
 [[  3   3]
 [  3 111]]


In [62]:

encoder = LabelEncoder()
rev_train['Pain_Relevance'] = encoder.fit_transform(data['Pain_Relevance'].astype(str))

#y = data[['Pain_Relevance']]
y = np.array(rev_train.Pain_Relevance)

# ML classification using only LDA topics as features

In [63]:
#For ML classification built from LDA topics alone

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1, cv_dt_f1, cv_mlp_f1, cv_rf_f1, cv_svm_f1  = [], [], [], [], [], [], []
cv_lr_precision, cv_lrsgd_precision, cv_svcsgd_precision, cv_dt_precision, cv_mlp_precision, cv_rf_precision, cv_svm_precision = [], [], [], [],[], [], []
cv_lr_recall, cv_lrsgd_recall, cv_svcsgd_recall, cv_dt_recall, cv_mlp_recall, cv_rf_recall, cv_svm_recall = [], [], [], [],[], [],[]


for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_lr_precision.append(precision_score(y_val, y_pred, average='binary'))
    cv_lr_recall.append(recall_score(y_val, y_pred, average='binary'))
    
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_lrsgd_precision.append(precision_score(y_val, y_pred, average='binary'))
    cv_lrsgd_recall.append(recall_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_svcsgd_precision.append(precision_score(y_val, y_pred, average='binary'))
    cv_svcsgd_recall.append(recall_score(y_val, y_pred, average='binary'))
    
    # Decision tree classifier
    dt = DecisionTreeClassifier(random_state=42).fit(X_train_scale, y_train)
    
    y_pred = dt.predict(X_val_scale)
    cv_dt_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_dt_precision.append(precision_score(y_val, y_pred, average='binary'))
    cv_dt_recall.append(recall_score(y_val, y_pred, average='binary'))
    
    # MLP Classifier
    mlp = MLPClassifier(
        hidden_layer_sizes=(5,5), max_iter=10000, random_state=42).fit(X_train_scale, y_train)
    y_pred = dt.predict(X_val_scale)
    cv_mlp_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_mlp_precision.append(precision_score(y_val, y_pred, average='binary'))
    cv_mlp_recall.append(recall_score(y_val, y_pred, average='binary'))
    
    #Random Forest Classifier
    rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42).fit(X_train_scale, y_train)
    
    y_pred = rf.predict(X_val_scale)
    cv_rf_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_rf_precision.append(precision_score(y_val, y_pred, average='binary'))
    cv_rf_recall.append(recall_score(y_val, y_pred, average='binary'))
    

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')
print(f'Decision Tree Val f1: {np.mean(cv_dt_f1):.3f} +- {np.std(cv_dt_f1):.3f}')
print(f'MLP Val f1: {np.mean(cv_mlp_f1):.3f} +- {np.std(cv_mlp_f1):.3f}')
print(f'Random Forest Val f1: {np.mean(cv_rf_f1):.3f} +- {np.std(cv_rf_f1):.3f}')

print('\n')


print(f'Logistic Regression Val precision: {np.mean(cv_lr_precision):.3f} +- {np.std(cv_lr_precision):.3f}')
print(f'Logisitic Regression SGD Val precision: {np.mean(cv_lrsgd_precision):.3f} +- {np.std(cv_lrsgd_precision):.3f}')
print(f'SVM Huber Val precision: {np.mean(cv_svcsgd_precision):.3f} +- {np.std(cv_svcsgd_precision):.3f}')
print(f'Decision Tree Val precision: {np.mean(cv_dt_precision):.3f} +- {np.std(cv_dt_precision):.3f}')
print(f'MLP Val precision: {np.mean(cv_mlp_precision):.3f} +- {np.std(cv_mlp_precision):.3f}')
print(f'Random Forest Val precision: {np.mean(cv_rf_precision):.3f} +- {np.std(cv_rf_precision):.3f}')

print('\n')
print(f'Logistic Regression Val recall: {np.mean(cv_lr_recall):.3f} +- {np.std(cv_lr_recall):.3f}')
print(f'Logisitic Regression SGD Val recall: {np.mean(cv_lrsgd_recall):.3f} +- {np.std(cv_lrsgd_recall):.3f}')
print(f'SVM Huber Val recall: {np.mean(cv_svcsgd_recall):.3f} +- {np.std(cv_svcsgd_recall):.3f}')
print(f'Decision Tree Val recall: {np.mean(cv_dt_recall):.3f} +- {np.std(cv_dt_recall):.3f}')
print(f'MLP Val recall: {np.mean(cv_mlp_recall):.3f} +- {np.std(cv_mlp_recall):.3f}')
print(f'Random Forest Val recall: {np.mean(cv_rf_recall):.3f} +- {np.std(cv_rf_recall):.3f}')


'''
print('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, dt_test_pred, average='weighted'))
print('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, dt_test_pred, average='macro'))
print('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, dt_test_pred, average='weighted'))

'''


Logistic Regression Val f1: 0.756 +- 0.042
Logisitic Regression SGD Val f1: 0.755 +- 0.141
SVM Huber Val f1: 0.977 +- 0.012
Decision Tree Val f1: 0.972 +- 0.013
MLP Val f1: 0.972 +- 0.013
Random Forest Val f1: 0.978 +- 0.011


Logistic Regression Val precision: 0.991 +- 0.010
Logisitic Regression SGD Val precision: 0.973 +- 0.034
SVM Huber Val precision: 0.955 +- 0.023
Decision Tree Val precision: 0.974 +- 0.017
MLP Val precision: 0.974 +- 0.017
Random Forest Val precision: 0.957 +- 0.022


Logistic Regression Val recall: 0.613 +- 0.054
Logisitic Regression SGD Val recall: 0.651 +- 0.223
SVM Huber Val recall: 1.000 +- 0.000
Decision Tree Val recall: 0.971 +- 0.010
MLP Val recall: 0.971 +- 0.010
Random Forest Val recall: 1.000 +- 0.000


"\nprint('Weighted Precision Score: %.3f' % metrics.precision_score(y2_test, dt_test_pred, average='weighted'))\nprint('Macro Recall Score: %.3f' % metrics.recall_score(y2_test, dt_test_pred, average='macro'))\nprint('Weighted Recall Score: %.3f' % metrics.recall_score(y2_test, dt_test_pred, average='weighted'))\n\n"